In [3]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
import random
import folium

# --- TTDI bounding box (approx) ---
north, south, east, west = 3.1505, 3.128, 101.641, 101.615

# --- Fetch plantable areas (grass, parks, meadows) from OSM ---
tags = {
    'leisure': 'park',
    'landuse': 'grass',
    'natural': 'grass'
}

bbox = (west, south, east, north)

try:
    plantable_areas = ox.features_from_bbox(bbox, tags)
    plantable_areas = plantable_areas.to_crs(epsg=3857)
    plantable_areas = plantable_areas[plantable_areas.geometry.notnull()]
    print(f"Found {len(plantable_areas)} plantable areas")
except Exception as e:
    print(f"Error fetching plantable areas: {e}")
    plantable_areas = gpd.GeoDataFrame(columns=['geometry'], geometry=[])

# --- Generate points inside plantable polygons ---
plantable_points = []

for poly in plantable_areas.geometry:
    if poly.is_empty:
        continue
    if poly.geom_type == 'Polygon':
        minx, miny, maxx, maxy = poly.bounds
        for _ in range(5):  # 5 random points per polygon
            x = random.uniform(minx, maxx)
            y = random.uniform(miny, maxy)
            pt = Point(x, y)
            if poly.contains(pt):
                plantable_points.append(pt)
    elif poly.geom_type == 'MultiPolygon':
        for subpoly in poly.geoms:
            minx, miny, maxx, maxy = subpoly.bounds
            for _ in range(5):
                x = random.uniform(minx, maxx)
                y = random.uniform(miny, maxy)
                pt = Point(x, y)
                if subpoly.contains(pt):
                    plantable_points.append(pt)

# --- Create GeoDataFrame for plantable points ---
plantable_points_gdf = gpd.GeoDataFrame(geometry=plantable_points, crs=plantable_areas.crs)
plantable_points_gdf = plantable_points_gdf.to_crs(epsg=4326)
print(f"Generated {len(plantable_points_gdf)} plantable points")

# --- Center map ---
center_lat = (north + south)/2
center_lon = (east + west)/2
print(f"Map center: {center_lat}, {center_lon}")

# --- Create Folium map ---
m = folium.Map(location=[center_lat, center_lon], zoom_start=16, tiles='CartoDB positron')

# Add bounding box for reference
# folium.Rectangle(
#     bounds=[[south, west], [north, east]],
#     color='red',
#     fill=False,
#     weight=2
# ).add_to(m)

# Add plantable points as blue dots
for _, row in plantable_points_gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        color="blue",
        fill=True,
        fill_opacity=0.7,
        popup="Plantable area"
    ).add_to(m)

# --- Save and display map ---
output_path = "/notebooks/ttdi_plantable_inside_grass.html"
m.save(output_path)
print(f"Map saved successfully at {output_path}")
m


Found 23 plantable areas
Generated 72 plantable points
Map center: 3.13925, 101.628
Map saved successfully at /notebooks/ttdi_plantable_inside_grass.html
